In [1]:
import logging
import sys
import warnings
from logging import StreamHandler
from os import chdir
from os.path import join
from subprocess import check_output
import pandas as pd
from google import auth

base_dir = check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode()
WORK_DIR = join(base_dir, 'src')
sys.path.extend([base_dir,
                join(base_dir, 'src'),
                join(base_dir, 'notebook'),
                join(base_dir, 'notebook/*')])

chdir(WORK_DIR)

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.8f}'.format

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(StreamHandler())

In [2]:
from data_providers.data_provider import DataProvider
from data_providers.tradingview_provider import TradingViewProvider

In [3]:
provider: DataProvider = TradingViewProvider()
provider.TOKEN = ''
provider.WORKERS_NO = 4

In [4]:
# symbols = [
#     'SP:S5COND',
# ]

# fields = [
#     'description'
#     'price',
#     'change_5d',
#     'change_5d_pct',
#     'low_5d',
#     'high_5d',
# ]

# results = provider.quotes(symbols=symbols, fields=fields)

In [5]:
symbols = [
    'BTCUSD',
    'SP:SPX',
    'NASDAQ:TSLA',
]

ohlcv = provider.ohlcv(symbols=symbols,
                       freq='1',
                       total_candles=100000,
                       tzinfo='America/Chicago').reset_index()

In [6]:
ohlcv

Field,Date,Symbol,Open,High,Low,Close,Volume
0,2025-02-09 18:01:00-06:00,BTCUSD,"96,573.00000000","96,608.00000000","96,535.00000000","96,593.00000000",1.37058625
1,2025-02-09 18:02:00-06:00,BTCUSD,"96,593.00000000","96,608.00000000","96,593.00000000","96,608.00000000",0.00221000
2,2025-02-09 18:03:00-06:00,BTCUSD,"96,578.00000000","96,750.00000000","96,578.00000000","96,750.00000000",6.73568595
3,2025-02-09 18:04:00-06:00,BTCUSD,"96,764.00000000","96,794.00000000","96,764.00000000","96,794.00000000",2.24935221
4,2025-02-09 18:05:00-06:00,BTCUSD,"96,794.00000000","96,817.00000000","96,745.00000000","96,817.00000000",1.26188033
...,...,...,...,...,...,...,...
36747,2025-02-20 14:55:00-06:00,NASDAQ:TSLA,353.38000000,353.94000000,353.23000000,353.81000000,"12,129.00000000"
36748,2025-02-20 14:56:00-06:00,NASDAQ:TSLA,353.82000000,354.32000000,353.82000000,354.32000000,"9,980.00000000"
36749,2025-02-20 14:57:00-06:00,NASDAQ:TSLA,354.27000000,354.61000000,354.10000000,354.37000000,"8,366.00000000"
36750,2025-02-20 14:58:00-06:00,NASDAQ:TSLA,354.41000000,354.57000000,354.34000000,354.55000000,"18,817.00000000"


In [7]:
ohlcv

Field,Date,Symbol,Open,High,Low,Close,Volume
0,2025-02-09 18:01:00-06:00,BTCUSD,"96,573.00000000","96,608.00000000","96,535.00000000","96,593.00000000",1.37058625
1,2025-02-09 18:02:00-06:00,BTCUSD,"96,593.00000000","96,608.00000000","96,593.00000000","96,608.00000000",0.00221000
2,2025-02-09 18:03:00-06:00,BTCUSD,"96,578.00000000","96,750.00000000","96,578.00000000","96,750.00000000",6.73568595
3,2025-02-09 18:04:00-06:00,BTCUSD,"96,764.00000000","96,794.00000000","96,764.00000000","96,794.00000000",2.24935221
4,2025-02-09 18:05:00-06:00,BTCUSD,"96,794.00000000","96,817.00000000","96,745.00000000","96,817.00000000",1.26188033
...,...,...,...,...,...,...,...
36747,2025-02-20 14:55:00-06:00,NASDAQ:TSLA,353.38000000,353.94000000,353.23000000,353.81000000,"12,129.00000000"
36748,2025-02-20 14:56:00-06:00,NASDAQ:TSLA,353.82000000,354.32000000,353.82000000,354.32000000,"9,980.00000000"
36749,2025-02-20 14:57:00-06:00,NASDAQ:TSLA,354.27000000,354.61000000,354.10000000,354.37000000,"8,366.00000000"
36750,2025-02-20 14:58:00-06:00,NASDAQ:TSLA,354.41000000,354.57000000,354.34000000,354.55000000,"18,817.00000000"


In [8]:
provider.calc_perf(ohlcv, '24h')

{'BTCUSD': {'close': 98492.0,
  'change_24h': 62.0,
  'change_24h_pct': 0.0006298892614040435,
  'low_24h': 98419.0,
  'high_24h': 98492.0},
 'SP:SPX': {'close': 6117.51,
  'change_24h': -0.4899999999997817,
  'change_24h_pct': -8.009153318074236e-05,
  'low_24h': 6117.51,
  'high_24h': 6118.0},
 'NASDAQ:TSLA': {'close': 354.4,
  'change_24h': 0.339999999999975,
  'change_24h_pct': 0.0009602892165169039,
  'low_24h': 352.77,
  'high_24h': 354.88}}

In [9]:
provider.calc_perf(ohlcv, '5D')

{'BTCUSD': {'close': 98492.0,
  'change_5d': 6.0,
  'change_5d_pct': 6.092236460004468e-05,
  'low_5d': 98456.0,
  'high_5d': 98492.0},
 'SP:SPX': {'close': 6117.51,
  'change_5d': -0.8000000000001819,
  'change_5d_pct': -0.0001307550614467364,
  'low_5d': 6116.36,
  'high_5d': 6119.53},
 'NASDAQ:TSLA': {'close': 354.4,
  'change_5d': 0.9899999999999523,
  'change_5d_pct': 0.0028012789677710087,
  'low_5d': 353.23,
  'high_5d': 354.61}}

In [10]:
provider.calc_perf(ohlcv, '1M')

{'BTCUSD': {'close': 98492.0,
  'change_1m': 68.0,
  'change_1m_pct': 0.0006908884012029586,
  'low_1m': 98419.0,
  'high_1m': 98492.0},
 'SP:SPX': {'close': 6117.51,
  'change_1m': 5.539999999999964,
  'change_1m_pct': 0.0009064180616069718,
  'low_1m': 6110.53,
  'high_1m': 6121.72},
 'NASDAQ:TSLA': {'close': 354.4,
  'change_1m': 1.3899999999999864,
  'change_1m_pct': 0.003937565508059223,
  'low_1m': 352.77,
  'high_1m': 354.88}}

In [11]:
provider.calc_perf(ohlcv, 'MTD')

{'BTCUSD': {'close': 98492.0,
  'change_mtd': nan,
  'change_mtd_pct': nan,
  'low_mtd': 93340.0,
  'high_mtd': 98871.0},
 'SP:SPX': {'close': 6117.51,
  'change_mtd': 76.98999999999978,
  'change_mtd_pct': 0.012745591439147585,
  'low_mtd': 5923.93,
  'high_mtd': 6147.43},
 'NASDAQ:TSLA': {'close': 354.4,
  'change_mtd': -50.10000000000002,
  'change_mtd_pct': -0.12385661310259585,
  'low_mtd': 325.1,
  'high_mtd': 394.0}}

In [12]:
provider.calc_perf(ohlcv, 'YTD')

{'BTCUSD': {'close': 98492.0,
  'change_ytd': nan,
  'change_ytd_pct': nan,
  'low_ytd': 93340.0,
  'high_ytd': 98871.0},
 'SP:SPX': {'close': 6117.51,
  'change_ytd': nan,
  'change_ytd_pct': nan,
  'low_ytd': 5774.42,
  'high_ytd': 6147.43},
 'NASDAQ:TSLA': {'close': 354.4,
  'change_ytd': nan,
  'change_ytd_pct': nan,
  'low_ytd': 325.1,
  'high_ytd': 439.72}}